In [83]:
import datetime
import os
import random
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy
import statsmodels.api as sm
import pylab as py

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.linear_model import LinearRegression

import math
import time
import datetime
from dython.nominal import associations
from dython.nominal import identify_nominal_columns

In [84]:
data = pd.read_csv("../../dataset/final_bike_sharing.csv")

In [85]:
cols = ['tripduration','gender','dist','hour','birthyear','usertype']

In [86]:
df = data[cols]

## Before Imputation the distributaion is 

In [87]:
# for col in  ['tripduration','gender','dist','hour','birthyear']:
#     np.random.seed(19680801)
#     plt.figure(figsize=(20,8))
#     mu = df[col].mean()
#     sigma = df[col].std()
#     x = df[col]
#     num_bins = 500
#     fig, ax = plt.subplots()
#     # the histogram of the data
#     n, bins, patches = ax.hist(x, num_bins, density=True)
#     # add a 'best fit' line
#     y = ((1 / (np.sqrt(2 * np.pi) * sigma)) *
#          np.exp(-0.5 * (1 / sigma * (bins - mu))**2))
#     ax.plot(bins, y, '--')
#     ax.set_xlabel(col)
#     ax.set_ylabel('Probability density')
#     ax.set_title(f'Histogram of IQ: {mu}, {sigma}')
#     plt.plot([mu,mu],[0, 2], lw=1.8, label="mean")
# #     plt.plot([sigma,sigma],[0, 2], lw=1.8, label="std")
# #     ax.set_ylim([0,2.5])
# #     ax.set_xlim([0, 6])
#     # Tweak spacing to prevent clipping of ylabel
#     fig.tight_layout()
#     plt.show()

In [88]:
# for col in df.columns:
#     norm_cdf = scipy.stats.norm.cdf(df[col]) # calculate the cdf - also discrete
#     # plot the cdf
#     sns.lineplot(x=df[col], y=norm_cdf)
#     plt.show()
#     sm.qqplot(df[col], line ='45')
#     py.show()


In [89]:
for col in  ['tripduration','dist','hour','birthyear']:
    mu = df[col].mean()
    sigma = df[col].std()
    print(f"The variable {col} has mean = {mu} and standard deviation ={sigma} ")
print("usertype mode is : " ,df['usertype'].mode() )
print("gender mode is : " , df['gender'].mode() )

The variable tripduration has mean = 525.9314525266259 and standard deviation =731.0970649533257 
The variable dist has mean = 0.7842291293398377 and standard deviation =0.5421989613484405 
The variable hour has mean = 13.633500453206436 and standard deviation =5.156725828688158 
The variable birthyear has mean = 1980.439108882846 and standard deviation =10.094119857977768 
usertype mode is :  0    Subscriber
Name: usertype, dtype: object
gender mode is :  0    male
Name: gender, dtype: object


In [90]:
def split_dataset(df):
    X = df.drop('tripduration', axis=1)
    y = df['tripduration']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print('Train', X_train.shape, y_train.shape)
    print('Test', X_test.shape, y_test.shape)
    return X_train, X_test, y_train, y_test

In [91]:
## get the sample size such as 5%, 10%,20% etc
def get_sample_size(df, perc):
    return round(perc/100 * df.shape[0])

def get_an_index(index):
    li = []
    for i in index:
        li.append(i)
    return li   

def ampute_nan_values(X_train,col_name, perc):
    random.seed(100)
    index = get_an_index(X_train.index)
    sample_size = get_sample_size(X_train, perc)
    selected_index = random.sample(index, sample_size)
    for i in selected_index:
        if col_name in X_train.columns:
            X_train.at[i,col_name]= np.NaN
        else:
            X_train.loc[i] = np.NaN
    return X_train

###====================####
def convert_to_dataframe(y_train):
    return pd.DataFrame(y_train)

def convert_to_series(y_train):
    return y_train.squeeze()

def ampute_nan_values_drop(X_train,y_train,col_name, perc):
    random.seed(100)
    index = get_an_index(X_train.index)
    sample_size = get_sample_size(X_train, perc)
    selected_index = random.sample(index, sample_size)
    y_train = convert_to_dataframe(y_train)
    for i in selected_index:
        if col_name in X_train.columns:
            X_train.at[i,col_name]= np.NaN
        else:
            X_train.loc[i] = np.NaN
        y_train.loc[i] = np.NaN
    y_train = convert_to_series(y_train)
    return X_train, y_train

In [92]:
def sample_imputation_X(var,X_train,X_test):
    # extract a random sample
    random_sample_train = X_train[var].dropna().sample(X_train[var].isnull().sum(), random_state=0)
    random_sample_test = X_test[var].dropna().sample(X_test[var].isnull().sum(), random_state=0)
     # re-index the randomly extracted sample
    random_sample_train.index = X_train[X_train[var].isnull()].index
    random_sample_test.index = X_test[X_test[var].isnull()].index

    # replace the NA
    X_train.loc[X_train[var].isnull(), var] = random_sample_train
    X_test.loc[X_test[var].isnull(), var] = random_sample_test
    return X_train, X_test

In [93]:
def one_hot_encoding(X_train,X_test):
    categorical_features=identify_nominal_columns(X_train)
    print(categorical_features)
    one_hot_encoded_data_train = pd.get_dummies(X_train, columns = categorical_features)
    one_hot_encoded_data_test = pd.get_dummies(X_test, columns = categorical_features)
    return one_hot_encoded_data_train,one_hot_encoded_data_test

In [94]:
def model_evaluation(X_train,y_train):
    print(datetime.datetime.now())
    model_lr = LinearRegression(fit_intercept=True).fit(X_train, y_train) 
    print(datetime.datetime.now())
    return model_lr

In [95]:
def ampute_each_variables(variable, method,X_train,y_train,X_test,y_test):
    if method == "remove":
        X_train , y_train = ampute_nan_values_drop(X_train,y_train,variable,perc=10)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("y_train Amputed value count of varaible ", y_train.isnull().sum())
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test , y_test = ampute_nan_values_drop(X_test,y_test,variable,perc=10)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("y_test Amputed value count of varaible ", y_test.isnull().sum())
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    else:
        X_train = ampute_nan_values(X_train,variable,perc=10)
        print('Train set after Amputation', X_train.shape)
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test = ampute_nan_values(X_test,variable,perc=10)
        print('Test set after Amputation', X_test.shape)
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    
    if method == "remove":
        print("Simulate Imputation for NA")
        X_train.dropna(axis=0, inplace=True)
        y_train.dropna(inplace=True)
        print('Train after NA value removal Imputation', X_train.shape, y_train.shape)
        X_test.dropna(axis=0, inplace=True)
        y_test.dropna(inplace=True)
        print('Test after NA value removal Imputation', X_test.shape, y_test.shape)
    if method == "mode":
        print("Simulate Imputation for Mode")
        X_train.fillna(X_train.mode().iloc[0], inplace=True)
        print('Train after Mode Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test.mode().iloc[0], inplace=True)
        print('Test after Mode Imputation', X_test.shape, y_test.shape)
    if method == "mean":
        print("Simulate Imputation for Mean")
        X_train.fillna(X_train.mean(), inplace=True)
        print('Train after Mean Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test.mean(), inplace=True)
        print('Test after Mean Imputation', X_test.shape, y_test.shape)
    if method == "sample":
        print("Simple Imputation using sampling")
        X_train, X_test = sample_imputation_X(variable, X_train, X_test)        
            
    return X_train , y_train, X_test , y_test

In [96]:
def visualize_dist(df, col):
    np.random.seed(19680801)
    plt.figure(figsize=(20,8))
    mu = df[col].mean()
    sigma = df[col].std()
    x = df[col]
    num_bins = 500
    fig, ax = plt.subplots()
    # the histogram of the data
    n, bins, patches = ax.hist(x, num_bins, density=True)
    # add a 'best fit' line
    y = ((1 / (np.sqrt(2 * np.pi) * sigma)) *
         np.exp(-0.5 * (1 / sigma * (bins - mu))**2))
    ax.plot(bins, y, '--')
    ax.set_xlabel(col)
    ax.set_ylabel('Probability density')
    ax.set_title(f'Histogram of IQ: {mu}, {sigma}')
    plt.plot([mu,mu],[0, 2], lw=1.8, label="mean")
    #     plt.plot([sigma,sigma],[0, 2], lw=1.8, label="std")
    #     ax.set_ylim([0,2.5])
    #     ax.set_xlim([0, 6])
    # Tweak spacing to prevent clipping of ylabel
    fig.tight_layout()
    plt.show()

In [97]:
features_for_amputation =['gender','dist','hour','birthyear','usertype']

## Calculate mean and std before imputation

In [98]:
X_train, X_test, y_train, y_test = split_dataset(df)

Train (282432, 5) (282432,)
Test (70608, 5) (70608,)


In [99]:
for col in  ['dist','hour','birthyear']:
    mu = X_train[col].mean()
    sigma = X_train[col].std()
    print(f"The variable {col} has mean = {mu} and standard deviation ={sigma} ")
    
print(f"The variable usertype categorical type mode is ", X_train['usertype'].mode() )    
print(f"The variable gender categorical type mode is ", X_train['gender'].mode() )    

The variable dist has mean = 0.7848814720776504 and standard deviation =0.54278813457282 
The variable hour has mean = 13.633274558123725 and standard deviation =5.155597162884377 
The variable birthyear has mean = 1980.450076478586 and standard deviation =10.101242979840942 
The variable usertype categorical type mode is  0    Subscriber
Name: usertype, dtype: object
The variable gender categorical type mode is  0    male
Name: gender, dtype: object


In [100]:
for col in ['dist','hour','birthyear']:
    mu = X_test[col].mean()
    sigma = X_test[col].std()
    print(f"The variable {col} has mean = {mu} and standard deviation ={sigma} ")
print(f"The variable usertype categorical type mode is ", X_test['usertype'].mode() )   
print(f"The variable gender categorical type mode is ", X_test['gender'].mode() ) 

The variable dist has mean = 0.7816197583886165 and standard deviation =0.539831786399696 
The variable hour has mean = 13.634404033537276 and standard deviation =5.161274457822756 
The variable birthyear has mean = 1980.3952384998868 and standard deviation =10.065528836178935 
The variable usertype categorical type mode is  0    Subscriber
Name: usertype, dtype: object
The variable gender categorical type mode is  0    male
Name: gender, dtype: object


## Imputation for NA ( drop NA values)

In [101]:
model_metrics ={}
cat_gender = ['gender_female', 'gender_male','gender_unknown']
cat_usertype = ['usertype_Customer', 'usertype_Subscriber']
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"remove",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    if(var == 'gender'):
        for v in cat_gender:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                           "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    elif(var == 'usertype'):
        for v in cat_usertype:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                            "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    else:
        mu_train = X_train[var].mean()
        sigma_train = X_train[var].std()
        mu_test = X_test[var].mean()
        sigma_test = X_test[var].std()
        model_metrics[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                              "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                               "intercept":model_reg.intercept_,
                              "Mu_train":mu_train, "sigma_train":sigma_train,
                              "Mu_test":mu_test, "sigma_test":sigma_test}
    
    
    del X_train
    del X_test
    del y_train
    del y_test

Now running model is .... gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5) (282432,)
y_train Amputed value count of varaible  28243
X_train Amputed value count of varaible  28243
Test set after Amputation (70608, 5) (70608,)
y_test Amputed value count of varaible  7061
X_test Amputed value count of varaible  7061
Simulate Imputation for NA
Train after NA value removal Imputation (254189, 5) (254189,)
Test after NA value removal Imputation (63547, 5) (63547,)
['gender', 'usertype']
2023-02-19 12:43:09.347869
2023-02-19 12:43:09.453890
Now running model is .... dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5) (282432,)
y_train Amputed value count of varaible  28243
X_train Amputed value count of varaible  28243
Test set after Amputation (70608, 5) (70608,)
y_test Amputed value count of varaible  7061
X_test Amputed value count of varaible  7061
Simulate Imputation for NA
Train after NA value

In [102]:
model_metrics

{'gender_female': {'MAE': 279.2542248801155,
  'MSE': 432806.1392976349,
  'RMSE': 657.880034122966,
  'R2': 0.16721358241898143,
  'intercept': 4330.683499886296,
  'Mu_train': 0.21332945170719425,
  'sigma_train': 0.40965919611487306,
  'Mu_test': 0.21078886493461532,
  'sigma_test': 0.40787196183577956},
 'gender_male': {'MAE': 279.2542248801155,
  'MSE': 432806.1392976349,
  'RMSE': 657.880034122966,
  'R2': 0.16721358241898143,
  'intercept': 4330.683499886296,
  'Mu_train': 0.7267899082965824,
  'sigma_train': 0.44560870578778683,
  'Mu_test': 0.7302154310982423,
  'sigma_test': 0.44385127623356097},
 'gender_unknown': {'MAE': 279.2542248801155,
  'MSE': 432806.1392976349,
  'RMSE': 657.880034122966,
  'R2': 0.16721358241898143,
  'intercept': 4330.683499886296,
  'Mu_train': 0.059880639996223284,
  'sigma_train': 0.23726603300852148,
  'Mu_test': 0.05899570396714243,
  'sigma_test': 0.2356185147712141},
 'dist': {'MAE': 279.2542248801155,
  'MSE': 432806.1392976349,
  'RMSE': 65

###  BootStarp estimation with drop amputation

In [103]:
bootstrap_data ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"remove",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx = np.arange(y_train.shape[0])
    bootstarp_train_accuracies = {}
    for i in range(501):
        train_idx = rng.choice(idx, size=idx.shape[0], replace=True)
        test_idx = np.setdiff1d(idx, train_idx, assume_unique=False)
        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]
        boot_test_X, boot_test_y = X_train.iloc[test_idx], y_train.iloc[test_idx]
        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}
                         
#     print(np.percentile(R2, 2.5), np.percentile(R2, 97.5))
#     print(np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5))
#     print(np.percentile(intercept, 2.5), np.percentile(intercept, 97.5))

Now running model is .... gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5) (282432,)
y_train Amputed value count of varaible  28243
X_train Amputed value count of varaible  28243
Test set after Amputation (70608, 5) (70608,)
y_test Amputed value count of varaible  7061
X_test Amputed value count of varaible  7061
Simulate Imputation for NA
Train after NA value removal Imputation (254189, 5) (254189,)
Test after NA value removal Imputation (63547, 5) (63547,)
['gender', 'usertype']
Now running model is .... dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5) (282432,)
y_train Amputed value count of varaible  28243
X_train Amputed value count of varaible  28243
Test set after Amputation (70608, 5) (70608,)
y_test Amputed value count of varaible  7061
X_test Amputed value count of varaible  7061
Simulate Imputation for NA
Train after NA value removal Imputation (254189, 5) (254189,)
Test after N

In [104]:
bootstrap_data

{'gender': {'R2': [0.1692903070566928, 0.18287391667440567],
  'RMSE': [649.4322232919918, 683.6458268938354],
  'intercept': [3702.721568724243, 4859.1833670179185]},
 'dist': {'R2': [0.1692903070566928, 0.18287391667440567],
  'RMSE': [649.4322232919918, 683.6458268938354],
  'intercept': [3702.721568724243, 4859.1833670179185]},
 'hour': {'R2': [0.1692903070566928, 0.18287391667440567],
  'RMSE': [649.4322232919918, 683.6458268938354],
  'intercept': [3702.721568724243, 4859.1833670179185]},
 'birthyear': {'R2': [0.1692903070566928, 0.18287391667440567],
  'RMSE': [649.4322232919918, 683.6458268938354],
  'intercept': [3702.721568724243, 4859.1833670179185]},
 'usertype': {'R2': [0.1692903070566928, 0.18287391667440567],
  'RMSE': [649.4322232919918, 683.6458268938354],
  'intercept': [3702.721568724243, 4859.1833670179185]}}

### Mode Replacement

In [105]:
model_metrics_mode ={}
for var in features_for_amputation:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mode",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    if(var == 'gender'):
        for v in cat_gender:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics_mode[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                            "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    elif(var == 'usertype'):
        for v in cat_usertype:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics_mode[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                           "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    else:
        mu_train = X_train[var].mean()
        sigma_train = X_train[var].std()
        mu_test = X_test[var].mean()
        sigma_test = X_test[var].std()
        model_metrics_mode[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                              "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                                "intercept":model_reg.intercept_,
                              "Mu_train":mu_train, "sigma_train":sigma_train,
                              "Mu_test":mu_test, "sigma_test":sigma_test}
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  28243
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  7061
Simulate Imputation for Mode
Train after Mode Imputation (282432, 5) (282432,)
Test after Mode Imputation (70608, 5) (70608,)
['gender', 'usertype']
2023-02-19 12:48:48.374735
2023-02-19 12:48:48.464297
Now model is running for varaible ==== dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  28243
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  7061
Simulate Imputation for Mode
Train after Mode Imputation (282432, 5) (282432,)
Test after Mode Imputation (70608, 5) (70608,)
['gender', 'usertype']
2023-02-19 12:48:49.793658
2023-02-19 12:48:49.877248
Now model is running for varaible ==== hour
Train (282432, 5) 

In [106]:
model_metrics_mode

{'gender_female': {'MAE': 278.68373476735826,
  'MSE': 429063.44288055133,
  'RMSE': 655.0293450529917,
  'R2': 0.16758965468078135,
  'intercept': 4407.742348775691,
  'Mu_train': 0.19199665760253795,
  'sigma_train': 0.3938711595850177,
  'Mu_test': 0.1897093813732155,
  'sigma_test': 0.39207385670261174},
 'gender_male': {'MAE': 278.68373476735826,
  'MSE': 429063.44288055133,
  'RMSE': 655.0293450529917,
  'R2': 0.16758965468078135,
  'intercept': 4407.742348775691,
  'Mu_train': 0.7541107239972807,
  'sigma_train': 0.43061397619134123,
  'Mu_test': 0.7571946521640607,
  'sigma_test': 0.42878143005567},
 'gender_unknown': {'MAE': 278.68373476735826,
  'MSE': 429063.44288055133,
  'RMSE': 655.0293450529917,
  'R2': 0.16758965468078135,
  'intercept': 4407.742348775691,
  'Mu_train': 0.05389261840018128,
  'sigma_train': 0.22580607745467474,
  'Mu_test': 0.05309596646272377,
  'sigma_test': 0.22422644106633338},
 'dist': {'MAE': 283.2296212487276,
  'MSE': 430551.94962635095,
  'RMSE

### Bootstarp for Mode Imputation

In [107]:
bootstrap_data_mode ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mode",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx = np.arange(y_train.shape[0])
    bootstarp_train_accuracies = {}
    for i in range(501):
        train_idx = rng.choice(idx, size=idx.shape[0], replace=True)
        test_idx = np.setdiff1d(idx, train_idx, assume_unique=False)
        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]
        boot_test_X, boot_test_y = X_train.iloc[test_idx], y_train.iloc[test_idx]
        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_mode[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}
#     print(np.percentile(R2, 2.5), np.percentile(R2, 97.5))
#     print(np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5))
#     print(np.percentile(intercept, 2.5), np.percentile(intercept, 97.5))

Now running model is .... gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  28243
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  7061
Simulate Imputation for Mode
Train after Mode Imputation (282432, 5) (282432,)
Test after Mode Imputation (70608, 5) (70608,)
['gender', 'usertype']
Now running model is .... dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  28243
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  7061
Simulate Imputation for Mode
Train after Mode Imputation (282432, 5) (282432,)
Test after Mode Imputation (70608, 5) (70608,)
['gender', 'usertype']
Now running model is .... hour
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  28243
Test set after Amputation 

In [108]:
bootstrap_data_mode

{'gender': {'R2': [0.16997660432429562, 0.1835356319143303],
  'RMSE': [649.8816105513088, 682.2198713448631],
  'intercept': [3843.377598410043, 4981.248931453127]},
 'dist': {'R2': [0.16762615966649053, 0.18079919850336795],
  'RMSE': [651.008253623839, 683.3143277562708],
  'intercept': [3558.304995363931, 4698.273353215009]},
 'hour': {'R2': [0.1700350649761721, 0.18345144505295674],
  'RMSE': [649.8888673399063, 682.2404490210199],
  'intercept': [3706.3446842892854, 4836.586634046061]},
 'birthyear': {'R2': [0.17024987025297977, 0.183650747144242],
  'RMSE': [649.764389445547, 682.1494081344799],
  'intercept': [3397.4268445384077, 4538.925440484303]},
 'usertype': {'R2': [0.15689657076436275, 0.17087599791601144],
  'RMSE': [655.2468635104944, 687.2619475365002],
  'intercept': [3432.090871497232, 4604.633226701961]}}

## Mean Imputation

In [109]:
variables =['dist','hour','birthyear']
model_metrics_mean ={}
for var in variables:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mean",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    mu_train = X_train[var].mean()
    sigma_train = X_train[var].std()
    mu_test = X_test[var].mean()
    sigma_test = X_test[var].std()
    model_metrics_mean[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                            "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}

    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  28243
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  7061
Simulate Imputation for Mean


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_28292\3010668567.py:35: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_train.fillna(X_train.mean(), inplace=True)


Train after Mean Imputation (282432, 5) (282432,)


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_28292\3010668567.py:37: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_test.fillna(X_test.mean(), inplace=True)


Test after Mean Imputation (70608, 5) (70608,)
['gender', 'usertype']
2023-02-19 12:59:24.982652
2023-02-19 12:59:25.079192
Now model is running for varaible ==== hour
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  28243
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  7061
Simulate Imputation for Mean


...

Test after Mean Imputation (70608, 5) (70608,)
['gender', 'usertype']
2023-02-19 13:03:21.840478
2023-02-19 13:03:21.935409
Now model is running for varaible ==== birthyear
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  28243
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  7061
Simulate Imputation for Mean


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_28292\3010668567.py:35: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_train.fillna(X_train.mean(), inplace=True)


Train after Mean Imputation (282432, 5) (282432,)


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_28292\3010668567.py:37: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_test.fillna(X_test.mean(), inplace=True)


Test after Mean Imputation (70608, 5) (70608,)
['gender', 'usertype']
2023-02-19 13:07:01.381347
2023-02-19 13:07:01.461999


In [110]:
model_metrics_mean

{'dist': {'MAE': 282.7806168658308,
  'MSE': 430535.91508764157,
  'RMSE': 656.1523566121222,
  'R2': 0.1647329650263376,
  'intercept': 4115.940493290295,
  'Mu_train': 0.7846721331167078,
  'sigma_train': 0.5151246269979137,
  'Mu_test': 0.7823161180295614,
  'sigma_test': 0.512367863719376},
 'hour': {'MAE': 278.6932198006469,
  'MSE': 429028.8170143051,
  'RMSE': 655.0029137448971,
  'R2': 0.16765683106170426,
  'intercept': 4257.934127420233,
  'Mu_train': 13.636136890269404,
  'sigma_train': 4.891218447910462,
  'Mu_test': 13.620910507182153,
  'sigma_test': 4.896798809282101},
 'birthyear': {'MAE': 278.6530299877195,
  'MSE': 429000.3044984794,
  'RMSE': 654.9811482008314,
  'R2': 0.16771214715431992,
  'intercept': 4313.150255488631,
  'Mu_train': 1980.4473836399623,
  'sigma_train': 9.58628267998128,
  'Mu_test': 1980.3931420840102,
  'sigma_test': 9.539770696393475}}

### Bootstrap for mean

In [111]:
bootstrap_data_mean = {}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mean",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx = np.arange(y_train.shape[0])
    bootstarp_train_accuracies = {}
    for i in range(501):
        train_idx = rng.choice(idx, size=idx.shape[0], replace=True)
        test_idx = np.setdiff1d(idx, train_idx, assume_unique=False)
        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]
        boot_test_X, boot_test_y = X_train.iloc[test_idx], y_train.iloc[test_idx]
        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_mean[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  28243
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  7061
Simulate Imputation for Mean


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_28292\3010668567.py:35: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_train.fillna(X_train.mean(), inplace=True)


Train after Mean Imputation (282432, 5) (282432,)


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_28292\3010668567.py:37: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_test.fillna(X_test.mean(), inplace=True)


Test after Mean Imputation (70608, 5) (70608,)
['gender', 'usertype']
Now running model is .... dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  28243
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  7061
Simulate Imputation for Mean


...

Test after Mean Imputation (70608, 5) (70608,)
['gender', 'usertype']
Now running model is .... usertype
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  28243
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  7061
Simulate Imputation for Mean


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_28292\3010668567.py:35: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_train.fillna(X_train.mean(), inplace=True)


Train after Mean Imputation (282432, 5) (282432,)


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_28292\3010668567.py:37: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_test.fillna(X_test.mean(), inplace=True)


Test after Mean Imputation (70608, 5) (70608,)
['gender', 'usertype']


In [112]:
bootstrap_data_mean

{'gender': {'R2': [0.1700662638910192, 0.18357051996191937],
  'RMSE': [649.8484696300372, 682.1920598357341],
  'intercept': [3797.3718626481427, 4923.946610684054]},
 'dist': {'R2': [0.16778358892726325, 0.18086761886874342],
  'RMSE': [650.9444334981852, 683.2606155151748],
  'intercept': [3559.4347627576713, 4706.78902112447]},
 'hour': {'R2': [0.170165103621014, 0.1835192990876529],
  'RMSE': [649.8157238910101, 682.1865055773817],
  'intercept': [3712.7564578599354, 4846.451810941811]},
 'birthyear': {'R2': [0.170281513662499, 0.18374532332964283],
  'RMSE': [649.7480810186221, 682.1293325139047],
  'intercept': [3710.858051283004, 4951.740677792273]},
 'usertype': {'R2': [0.1576918683829046, 0.17160917330829317],
  'RMSE': [654.963145652195, 686.9468830945938],
  'intercept': [2989.2880572956933, 4173.116349938559]}}

## Sampling Replacement Imputation

In [113]:
model_metrics_sample ={}
for var in features_for_amputation:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"sample",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    if(var == 'gender'):
        for v in cat_gender:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics_sample[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                            "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    elif(var == 'usertype'):
        for v in cat_usertype:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics_sample[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                            "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    else:
        mu_train = X_train[var].mean()
        sigma_train = X_train[var].std()
        mu_test = X_test[var].mean()
        sigma_test = X_test[var].std()
        model_metrics_sample[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                              "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                               "intercept":model_reg.intercept_,
                              "Mu_train":mu_train, "sigma_train":sigma_train,
                              "Mu_test":mu_test, "sigma_test":sigma_test}
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  28243
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  7061
Simple Imputation using sampling
['gender', 'usertype']
2023-02-19 13:27:35.294936
2023-02-19 13:27:35.366505
Now model is running for varaible ==== dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  28243
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  7061
Simple Imputation using sampling
['gender', 'usertype']
2023-02-19 13:27:36.317276
2023-02-19 13:27:36.378277
Now model is running for varaible ==== hour
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  28243
Test set after Amputation (70608, 5)
X_test Amputed value count o

In [114]:
model_metrics_sample

{'gender_female': {'MAE': 278.76076998898355,
  'MSE': 429156.02409157454,
  'RMSE': 655.100010755285,
  'R2': 0.16741004124804404,
  'intercept': 4517.511059910166,
  'Mu_train': 0.2130813788805801,
  'sigma_train': 0.4094854070041289,
  'Mu_test': 0.21116587355540448,
  'sigma_test': 0.408138709980046},
 'gender_male': {'MAE': 278.76076998898355,
  'MSE': 429156.02409157454,
  'RMSE': 655.100010755285,
  'R2': 0.16741004124804404,
  'intercept': 4517.511059910166,
  'Mu_train': 0.7269077158395649,
  'sigma_train': 0.44554864085151974,
  'Mu_test': 0.7298889644232948,
  'sigma_test': 0.44402010795404767},
 'gender_unknown': {'MAE': 278.76076998898355,
  'MSE': 429156.02409157454,
  'RMSE': 655.100010755285,
  'R2': 0.16741004124804404,
  'intercept': 4517.511059910166,
  'Mu_train': 0.060010905279854976,
  'sigma_train': 0.23750746568501435,
  'Mu_test': 0.0589451620213007,
  'sigma_test': 0.23552370479563514},
 'dist': {'MAE': 286.45448848217734,
  'MSE': 432069.9376809045,
  'RMSE':

### Bootstarp for Sample

In [115]:
bootstrap_data_sample ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"sample",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx = np.arange(y_train.shape[0])
    bootstarp_train_accuracies = {}
    for i in range(501):
        train_idx = rng.choice(idx, size=idx.shape[0], replace=True)
        test_idx = np.setdiff1d(idx, train_idx, assume_unique=False)
        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]
        boot_test_X, boot_test_y = X_train.iloc[test_idx], y_train.iloc[test_idx]
        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_sample[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  28243
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  7061
Simple Imputation using sampling
['gender', 'usertype']
Now running model is .... dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  28243
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  7061
Simple Imputation using sampling
['gender', 'usertype']
Now running model is .... hour
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  28243
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  7061
Simple Imputation using sampling
['gender', 'usertype']
Now running model is .... birthyear
Train (282432, 5) (282432,)
Test (7060

In [116]:
bootstrap_data_sample

{'gender': {'R2': [0.1698543806696846, 0.18332535945308626],
  'RMSE': [649.9251616738643, 682.275451812396],
  'intercept': [3931.1548116314098, 5096.4652742285925]},
 'dist': {'R2': [0.16537963764702812, 0.17849683671605704],
  'RMSE': [651.9672195243456, 684.1042671968746],
  'intercept': [3444.6521142479487, 4598.230480736644]},
 'hour': {'R2': [0.17001266884351496, 0.18337376534128502],
  'RMSE': [649.8948303836703, 682.2323406450054],
  'intercept': [3703.991486917915, 4838.313142447543]},
 'birthyear': {'R2': [0.1702223209679128, 0.18367958755592245],
  'RMSE': [649.7698811628004, 682.1455282397708],
  'intercept': [3370.6060523439673, 4557.4314102041135]},
 'usertype': {'R2': [0.148283525129633, 0.16136721499136536],
  'RMSE': [659.0175933284797, 691.0306534521352],
  'intercept': [3142.625196729451, 4324.445531734245]}}